In [ ]:
import numpy as np
import pandas as pd
import random
import pandas as pd
import pysam
import os

def kmer2seq(kmers):
    """
    Convert kmers to original sequence
    
    Arguments:
    kmers -- str, kmers separated by space.
    
    Returns:
    seq -- str, original sequence.

    """
    kmers_list = kmers.split(" ")
    bases = [kmer[0] for kmer in kmers_list[0:-1]]
    bases.append(kmers_list[-1])
    seq = "".join(bases)
    assert len(seq) == len(kmers_list) + len(kmers_list[0]) - 1
    return seq

def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space

    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers


def get_train_valid_idx(sample_num):
    """
    Obtain the indices for the training set and validation set according to an 8:2 ratio
    """
    train_idx = []
    valid_idx = []
    random_idx = list(range(sample_num))
    random.shuffle(random_idx)
    avg_group_count = 10
    step = int(len(random_idx)/avg_group_count)
    random_idx_group = [random_idx[i:i+step] for i in range(0,len(random_idx),step)]
    for i in range(8):
        train_idx.extend(random_idx_group[i])
    for i in range(8,len(random_idx_group)):
        valid_idx.extend(random_idx_group[i])
    return {'train':train_idx,'valid':valid_idx}


# Generate train set and validation set

Generate and save the training and validation datasets to the "train_valid_csv" directory based on the "fa_diff_len" directory.

In [ ]:

fasta_dir = "./fa_diff_len/"
train_data_dir = "./train_valid_tsv"
os.makedirs(train_data_dir,exist_ok=True)
for specie in ['human','pig']:
    if specie=='pig':
        extend_lens = [64,128,256]
        cre_types = ['e1','e4e5','e10','e1_e4e5_e10']
    elif specie=="human":
        extend_lens = [64,128,175,256]
        cre_types = ['promoter','enhancer','promoter_enhancer']
    for extend_len in extend_lens:
        for cre_type in cre_types:
            train_dir = f"{cre_type}_{extend_len*2+1}"
            absolute_dir = os.path.join(train_data_dir,specie,train_dir)
            os.mkdir(absolute_dir)
            for train_valid in ['train','valid']:
                output_tsv = "train.tsv" if train_valid=="train" else "dev.tsv"
                tsv_path = os.path.join(absolute_dir,output_tsv)
                seqs = {}
                kmers_label = {}
                for pos_neg in ['positive','negative']:
                    seq_label = 1 if pos_neg=="positive" else 0
                    fafile = os.path.join(fasta_dir,specie,f"{cre_type}_len_{extend_len*2+1}_{train_valid}_{pos_neg}.fa")
                    fa = pysam.FastaFile(fafile)
                    seqs[pos_neg] = [fa.fetch(rederence) for rederence in fa.references]
                    kmers_label[pos_neg] = [(seq2kmer(seq,k=6),seq_label) for seq in seqs[pos_neg]]
                merge_sample = kmers_label['positive']+kmers_label['negative']
                random.shuffle(merge_sample)
                df_tsv = pd.DataFrame(merge_sample)
                df_tsv.columns = ['sequence','label']
                df_tsv.to_csv(tsv_path,sep="\t",index=None)

# Generate model fine-tuning script

In [ ]:
def get_cmd(data_path,output_path,cuda_divice,seq_len,loggin_steps,save_steps,batch_size,n_process=32):

    bashFile = f"""

    export KMER=6
    export MODEL_PATH=/data2/zyd_workspace/2024_10_pig_AS_SNV/DNABERT/6-new-12w-0
    export DATA_PATH={data_path}
    export OUTPUT_PATH={output_path}

    CUDA_VISIBLE_DEVICES={cuda_divice} python run_finetune.py --model_type dna --tokenizer_name=dna$KMER --model_name_or_path $MODEL_PATH --task_name dnaprom --do_train --do_eval --data_dir $DATA_PATH --max_seq_length {seq_len} --per_gpu_eval_batch_size={batch_size}   --per_gpu_train_batch_size={batch_size}   --learning_rate 5e-5 --num_train_epochs 20 --output_dir $OUTPUT_PATH --evaluate_during_training --logging_steps {loggin_steps} --save_steps {save_steps} --warmup_percent 0.1 --hidden_dropout_prob 0.1 --overwrite_output --weight_decay 0.01 --n_process {n_process}

    """
    return bashFile


In [ ]:


batch_size = 32

train_data_dir = "./train_valid_tsv"
train_output_dir = "./train_output"
os.makedirs(train_output_dir,exist_ok=True)
all_train_cmd = []
for specie in ['human','pig']:
    if specie=='pig':
        extend_lens = [256,128,64]
        cre_types = ['e1','e4e5','e10','e1_e4e5_e10']
    elif specie=="human":
        extend_lens = [256,175,128,64]
        cre_types = ['promoter','enhancer','promoter_enhancer']
    for extend_len in extend_lens:
        for cre_type in cre_types:
            train_dir = f"{cre_type}_{extend_len*2+1}"
            data_path = os.path.join(train_data_dir,specie,train_dir)
            output_path = os.path.join(train_output_dir,specie,train_dir)
            df_train_tsv = pd.read_csv(os.path.join(data_path,"train.tsv"),sep="\t")
            loggin_steps = int(df_train_tsv.shape[0]/batch_size)
            save_steps = int(df_train_tsv.shape[0]/batch_size)
            seq_len = extend_len*2+1-6+1+2
            cuda_divice = 1 if extend_len<175 else 0
            all_train_cmd.append(get_cmd(data_path,output_path,cuda_divice,seq_len,loggin_steps,save_steps,batch_size,n_process=32))

In [ ]:
f_4090_1 = open("./train_4090_1.sh","w")
f_4090_2 = open("./train_4090_2.sh","w")
for i in all_train_cmd:
    if "CUDA_VISIBLE_DEVICES=0" in i:
        f_4090_1.write(f"{i}\n")
    else:
        f_4090_2.write(f"{i}\n")
f_4090_1.close()
f_4090_2.close()